In [111]:
import pandas as pd 
import requests
from config import API_KEY
import os

In [115]:
# Create a list of coordinates that separate Melbourne CBD into a 5x5 grid
NORTHEAST_LAT = -37.804372697115774
NORTHEAST_LNG = 144.9738931822595
SOUTHWEST_LAT = -37.82082971518847  
SOUTHWEST_LNG = 144.9551814533306  
-37.804372697115774, 144.9738931822595
DESIRED_GRID_LENGTH = 16  # start with 3 to get a 3x3 grid and so on...

grid_coordinates = []
epsilon = 1e-9  # because float comparisons can be imprecise
intermediate_grid_length = DESIRED_GRID_LENGTH - 1

lat_step_size = (NORTHEAST_LAT - SOUTHWEST_LAT) / intermediate_grid_length
lng_step_size = (NORTHEAST_LNG - SOUTHWEST_LNG) / intermediate_grid_length

lat = SOUTHWEST_LAT
while lat <= NORTHEAST_LAT + epsilon:
    lng = SOUTHWEST_LNG
    while lng <= NORTHEAST_LNG + epsilon:
        grid_coordinates.append((lat, lng))
        lng += lng_step_size
    lat += lat_step_size

print(grid_coordinates)


[(-37.82082971518847, 144.9551814533306), (-37.82082971518847, 144.95642890192585), (-37.82082971518847, 144.9576763505211), (-37.82082971518847, 144.95892379911635), (-37.82082971518847, 144.9601712477116), (-37.82082971518847, 144.96141869630685), (-37.82082971518847, 144.9626661449021), (-37.82082971518847, 144.96391359349735), (-37.82082971518847, 144.9651610420926), (-37.82082971518847, 144.96640849068785), (-37.82082971518847, 144.9676559392831), (-37.82082971518847, 144.96890338787836), (-37.82082971518847, 144.9701508364736), (-37.82082971518847, 144.97139828506886), (-37.82082971518847, 144.9726457336641), (-37.82082971518847, 144.97389318225936), (-37.81973258065029, 144.9551814533306), (-37.81973258065029, 144.95642890192585), (-37.81973258065029, 144.9576763505211), (-37.81973258065029, 144.95892379911635), (-37.81973258065029, 144.9601712477116), (-37.81973258065029, 144.96141869630685), (-37.81973258065029, 144.9626661449021), (-37.81973258065029, 144.96391359349735), (-3

In [119]:
# Google Place API Function
def google_maps_restaurant(api_key, grid_coordinates, radius=175, max_iterations=3, min_ratings=35):
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    restaurants = []

    for coord in grid_coordinates:  # At each grid coordinate, call google API 
        location = f"{coord[0]},{coord[1]}"
        npt = None
        iteration = 0

        while iteration < max_iterations:
            params = {
                'location': location,
                'radius': radius,
                'type': 'restaurant',
                'key': api_key,
            }
            if npt:  # Ensures first iteration does not error out (since npt = None first) + add page token to params for the next iteration
                params['pagetoken'] = npt

            response = requests.get(url, params=params).json()

            if 'results' in response:  # If restaurant found
                for item in response['results']:
                    user_ratings_total = item.get('user_ratings_total', 0)
                    
                    if user_ratings_total >= min_ratings:
                        try: 
                            restaurant_info = {
                                'name': item['name'],
                                'place_id': item['place_id'],
                                'price_level': item.get('price_level', None),
                                'lat': item['geometry']['location']['lat'],
                                'long': item['geometry']['location']['lng'],
                                'types': item['types'],
                                'address': item['vicinity']
                            }

                            restaurants.append(restaurant_info)
                        except KeyError as e:
                            print(f"Error occurred while processing item data: {e}")
                        
            if 'next_page_token' in response:
                npt = response['next_page_token']
                iteration += 1
            else:
                break

    return restaurants

In [120]:
# Calls API - CARE FOR API COSTS
restaurant_list = google_maps_restaurant(API_KEY, grid_coordinates)

In [121]:
restaurant_list

[{'name': 'Best Western Melbourne City Hotel',
  'place_id': 'ChIJWXdjjVFd1moRh_LdkkyRy5s',
  'price_level': None,
  'lat': -37.8206276,
  'long': 144.9551461,
  'types': ['cafe',
   'lodging',
   'restaurant',
   'food',
   'point_of_interest',
   'establishment'],
  'address': '16 Spencer Street, Melbourne'},
 {'name': 'Grand Hotel Melbourne',
  'place_id': 'ChIJ3dmt61Fd1moRKaWbEW3hml4',
  'price_level': None,
  'lat': -37.8203025,
  'long': 144.9543906,
  'types': ['lodging',
   'bar',
   'restaurant',
   'food',
   'point_of_interest',
   'establishment'],
  'address': '33 Spencer Street, Docklands'},
 {'name': "Domino's Pizza Flinders St",
  'place_id': 'ChIJzUeRGVJd1moR8OAx5QG2svk',
  'price_level': 1,
  'lat': -37.8208688,
  'long': 144.9563465,
  'types': ['meal_delivery',
   'meal_takeaway',
   'restaurant',
   'food',
   'point_of_interest',
   'establishment'],
  'address': 'Tenancy 2C/555 Flinders Street, Melbourne'},
 {'name': 'City Kebabs',
  'place_id': 'ChIJJaylGVJd1moR

In [122]:
print(len(restaurant_list))

3200


## Data Cleanup

In [123]:
restaurant_df = pd.DataFrame(restaurant_list)
print(restaurant_df.shape)
restaurant_df.head()


(3200, 7)


,name,place_id,price_level,lat,long,types,address
0,Best Western Melbourne City Hotel,ChIJWXdjjVFd1moRh_LdkkyRy5s,NaN,-37.820628,144.955146,"[cafe, lodging, restaurant, food, point_of_int...","16 Spencer Street, Melbourne"
1,Grand Hotel Melbourne,ChIJ3dmt61Fd1moRKaWbEW3hml4,NaN,-37.820302,144.954391,"[lodging, bar, restaurant, food, point_of_inte...","33 Spencer Street, Docklands"
2,Domino's Pizza Flinders St,ChIJzUeRGVJd1moR8OAx5QG2svk,1.0,-37.820869,144.956346,"[meal_delivery, meal_takeaway, restaurant, foo...","Tenancy 2C/555 Flinders Street, Melbourne"
3,City Kebabs,ChIJJaylGVJd1moRjyu89QjaLrg,1.0,-37.820527,144.956193,"[meal_takeaway, restaurant, food, point_of_int...","3/546 Flinders Street, Melbourne"
4,Royale PizzaMelbourne,ChIJT5at6FFd1moRl7tSBihHz1I,NaN,-37.819691,144.954892,"[meal_delivery, meal_takeaway, restaurant, foo...","44/56 Spencer Street, Melbourne"


In [124]:
# Remove duplicates
restaurant_df.drop_duplicates(subset=['place_id'],inplace=True)
restaurant_df.shape

(715, 7)

In [125]:
# Remove all 'restaurants' that have more than 6 types + remove hotel
filtered_df = restaurant_df[restaurant_df['types'].apply(lambda x: len(x) <= 6)]
filtered_df = filtered_df[~filtered_df['name'].apply(lambda x: 'Hotel' in x)]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.shape


(656, 7)

In [126]:
# Change price_level to int - fill NaNs with 0
filtered_df = filtered_df.fillna(0)
filtered_df['price_level'] = filtered_df['price_level'].astype(int)
filtered_df.head()

,name,place_id,price_level,lat,long,types,address
0,Domino's Pizza Flinders St,ChIJzUeRGVJd1moR8OAx5QG2svk,1,-37.820869,144.956346,"[meal_delivery, meal_takeaway, restaurant, foo...","Tenancy 2C/555 Flinders Street, Melbourne"
1,City Kebabs,ChIJJaylGVJd1moRjyu89QjaLrg,1,-37.820527,144.956193,"[meal_takeaway, restaurant, food, point_of_int...","3/546 Flinders Street, Melbourne"
2,Royale PizzaMelbourne,ChIJT5at6FFd1moRl7tSBihHz1I,0,-37.819691,144.954892,"[meal_delivery, meal_takeaway, restaurant, foo...","44/56 Spencer Street, Melbourne"
3,Cherry and Twigs,ChIJJbxbHVJd1moRlmxHO37SS70,2,-37.820037,144.955732,"[cafe, restaurant, food, point_of_interest, es...","Shop 1B/555 Flinders Lane, Melbourne"
4,Restaurant 1903,ChIJlVbz5VFd1moRBbVdFgoTEVI,1,-37.819733,144.954767,"[restaurant, food, point_of_interest, establis...","44 Spencer Street, Melbourne"


In [127]:
# Save!
output_folder = 'output'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


output_file = 'restaurants.csv'
output_path = os.path.join(output_folder, output_file)
filtered_df.to_csv(output_path, index=False)